In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd
import datetime
from functools import reduce

## Verification of the goodness of the data

In [2]:
years = ['2016-2017', '2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']


path = '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/'

sexes = ['men', 'women']

loops = {'sprint':['loop_1', 'loop_2'], 'mass_start': ['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
    'individual': ['loop_1', 'loop_2', 'loop_3', 'loop_4'], 'mass_start': ['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
        'pursuit':['loop_1', 'loop_2', 'loop_3', 'loop_4'], 'sprint_2':['loop_1', 'loop_2', 'loop_3', 'loop_4'],\
            'pursuit_2': ['loop_1', 'loop_2', 'loop_3', 'loop_4']}




In [3]:
i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:            
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/{format}/*.txt')
                for file in files:
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [4]:
i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                for loop in loops[format]:
                    files_loop = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/{loop}_{location}_{format}.txt')                
                for file in files_loop:
                    
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [5]:

i = 0
for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt')                
                for file in files_loop:
                    
                    try:
                        pd.read_csv(file, sep = '\t')
                    except:
                        i += 1
                        print(file, i)
                        

In [6]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oestersund/metadata_Oestersund_sprint.txt', sep = '\t', index_col = 0)

,30 min.\nbefore Start,at\nStart Time,30 min.\nafter Start,at\nEnd Time
Sky,Mostly Cloudy,Mostly Cloudy,Mostly Cloudy,Mostly Cloudy
Snow Condition,Compact,Compact,Compact,Compact
Snow Temperature (in °C),-2,-2,-2,-2
Air Temperature (in °C),-6,-5.8,-5.8,-6.3
Humidity (in %),79,77,78,78
Wind,SE / 0.3 m/s,SW / 0.4 m/s,S / 1.0 m/s,SW / 0.9 m/s


## Creation of the dataframe

In [7]:
re.findall('[0-9][0-9]*\.[0-9]', '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/13.5km.txt')

['13.5']

In [8]:
input_predictor = pd.DataFrame(columns=['Rank',
 'Bib',
 'Family\xa0Name',
 'Given Name',
 'Nation',
 'Sector Time',
 'Sector Rank',
 'Net Time',
 'Behind']) #Creating the DataFrame to host the input data

In [9]:
sexes

['men', 'women']

In [10]:
def find_distance(file):
    return float(re.findall('[0-9][0-9]*\.[0-9]', file)[0])

In [11]:
def to_seconds(string):
    string = str(string)
    splitted_string =  string.split(':')
    if len(splitted_string) == 1:
        return float(splitted_string[0])
    elif len(splitted_string) == 2:
        return float(splitted_string[0])*60 + float(splitted_string[1])

In [12]:
indexes_race = {'men':int(0), 'women':int(0)}
pattern = '[0-9][0-9]*\.[0-9]'


for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                indexes_race[sex] += 1        
                files = glob.glob(path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/{format}/*.txt')
                
                metadata_2 = pd.read_csv(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt', sep = '\t', index_col = 0)
                
                    
                    
                metadata_2 = metadata_2.T
                distances = list(map(find_distance, files))
                indexes = np.argsort(np.array(distances))
                files = np.array(files)[indexes]
                for file in files:
                    with open(file, mode = 'r') as f_0:
                        if f_0.readlines()[1] == 'Behind\n':
                            with open(file, mode = 'r') as f:
                        
                                lines = [line for line in f.readlines()]
                                if lines[1] == 'Behind\n':
                                    with open(file, mode = 'w') as f_2:
                                        line_1 = lines[0][:-1] + 'Behind\n'
                                        f_2.write(line_1 + reduce(lambda x,y: x + y, lines[2:]))
                    
                    #display(metadata_2)
                    df = pd.read_csv(file, sep='\t')
                    distance = re.findall(pattern, file)[0]
                    df['sex'] = np.array([sex]*len(df))
                    df['distance'] = np.array([distance]*len(df))
                    df['race'] = np.array([indexes_race[sex]]*len(df))
                    df['source'] = np.array(['intermediate']*len(df))
                    df['format'] = np.array([format]*len(df))
                    df['season'] = np.array([year]*len(df))
                    
                    df['date'] = metadata_2['Date'][0]

                    
                    
                    try:
                        df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
                    except:
                        print('WARNINGGGGGG', file) 

                    df['Behind'] = np.array((map(to_seconds, df['Behind'])))
                
                    idx_sort_behind = df['Behind'].argsort().argsort()
                

                    df['position'] = idx_sort_behind + 1
                    df['venue'] = np.array([location]*len(df))
                

                
                    input_predictor = pd.concat([input_predictor, df], axis = 0)
                    

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Behind'][df['Behind'] == '+2:43.3zy'] = '+2:43.3'
/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2831751344.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [13]:
#input_predictor.to_csv('predictor_df.csv', index=False)

## Arrichment of the dataframe

In [14]:


with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Kontiolahti/intermediates/pursuit/3.6km.txt', mode = 'r') as f:
    
    lines = [line for line in f.readlines()]
    if lines[1] == 'Behind\n':
        with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Kontiolahti/intermediates/pursuit/3.6km.txt', mode = 'w') as f_2:
            line_1 = lines[0][:-1] + 'Behind\n'
            f_2.write(line_1 + reduce(lambda x,y: x + y, lines[2:]))

In [15]:
loops_race = {'sprint': 2, 'pursuit': 4, 'mass_start': 4, 'individual': 4, 'sprint_2': 2, 'pursuit_2': 4}
race_length = {'men' : {'sprint': 10, 'pursuit': 12.5, 'mass_start': 15, 'individual': 20,\
    'sprint_2': 10, 'pursuit_2':12.5}, 'women' : {'sprint': 7.5, 'pursuit': 10, 'mass_start': 12.5, 'individual': 15,\
        'sprint_2': 7.5, 'pursuit_2':10}}

In [16]:
with open('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2014-2015_m/Anterselva/loop_3_Anterselva_sprint.txt', 'r') as f:
    columns_loop = f.readlines()[0]
    
columns_loop = columns_loop.split('\t')
columns_loop

['Rank',
 'Bib',
 'Family\xa0Name',
 'Given Name',
 'Nation',
 'Cumulative Time',
 'Loop Time',
 'Course Time\n']

In [17]:
def to_seconds_loop(string):
    string = str(string)
    if '+' in string:
        return to_seconds(string)
    else:
        return 0

In [18]:
indexes_race = {'men':int(0), 'women':int(0)}
df_loops = pd.DataFrame(columns = columns_loop)

for year in years:
    for sex in sexes:
        locations = os.listdir(path + f'{sex}/{year}_{sex[0]}')
        locations = np.array(locations)[np.array(locations) !=  '.DS_Store']
        for location in locations:
            format_path = glob.glob((path + f'{sex}/{year}_{sex[0]}/{location}/intermediates/*'))
            formats = [f.split('/')[-1] for f in format_path]
            for format in formats:
                indexes_race[sex] += 1        
                for loop_number in range(loops_race[format]):
                    file = path + f'{sex}/{year}_{sex[0]}/{location}/loop_{loop_number + 1}_{location}_{format}.txt'
                    
                    df = pd.read_csv(file, sep = '\t')
                    metadata_2 = pd.read_csv(path + f'{sex}/{year}_{sex[0]}/{location}/metadata_2_{location}_{format}.txt', sep = '\t', index_col = 0)
                    metadata_2 = metadata_2.T

                    df['distance'] = [(race_length[sex][format]/loops_race[format])*(loop_number + 1)]*len(df)
                    df['sex'] = [sex]*len(df)
                    df['date'] = metadata_2['Date'][0]
                    df['race'] = np.array([indexes_race[sex]]*len(df))
                    df['source'] = np.array(['loop']*len(df))
                    df['format'] = np.array([format]*len(df))

                    df['Behind'] = np.array((map(to_seconds_loop, df['Cumulative Time'])))
                    df['season'] = np.array([year]*len(df))
                    df['venue'] = np.array([location]*len(df))
                
                    idx_sort_behind = df['Behind'].argsort().argsort()
                

                    df['position'] = idx_sort_behind + 1
                
                    df_loops = pd.concat([df_loops, df], axis = 0)
                    

In [19]:
#df_loops.to_csv('loops_df.csv', index=False)

#--------------------------------------------#

<img src="start_here.jpeg" alt="">

In [20]:
df_predictor = pd.read_csv('predictor_df.csv')
df_predictor = df_predictor.astype({'race': 'int32'})
df_predictor.head()

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/3464265586.py:1: DtypeWarning: Columns (0,1,6,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_predictor = pd.read_csv('predictor_df.csv')


,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,race,source,format,season,date,position,venue,vRank,Unnamed: 0,Day Time
0,1,3,Schempp,Simon,GER,2:10.1,4.0,2:10.1,1.2,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,4.0,Oberhof,NaN,NaN,NaN
1,2,6,Lesser,Erik,GER,2:11.0,7.0,2:11.0,2.1,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,7.0,Oberhof,NaN,NaN,NaN
2,3,1,Fourcade,Martin,FRA,2:08.9,1.0,2:08.9,0.0,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,1.0,Oberhof,NaN,NaN,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,2:15.5,23.0,2:15.5,6.6,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,22.0,Oberhof,NaN,NaN,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,2:11.3,8.0,2:11.3,2.4,men,...,1,intermediate,mass_start,2016-2017,2017-01-08,8.0,Oberhof,NaN,NaN,NaN


In [21]:
file = '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Kontiolahti/intermediates/pursuit/4.3km.txt'

with open(file, mode = 'r') as f_0:
    
    if f_0.readlines()[1] == 'Behind\n':
        
        with open(file, mode = 'r') as f:
    
            lines = [line for line in f.readlines()]
            if lines[1] == 'Behind\n':
                
                with open(file, mode = 'w') as f_2:
                    line_1 = lines[0][:-1] + 'Behind\n'
                    f_2.write(line_1 + reduce(lambda x,y: x + y, lines[2:]))
                    

In [22]:
df_loops = pd.read_csv('loops_df.csv')

In [23]:
df_loops.reset_index(inplace = True, drop = True)
df_loops['race'] = df_loops['race'].astype({'race': 'int32'})

df_loops.head() 

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time\n,Course Time,Range Time,...,sex,date,race,source,format,Behind,season,venue,position,Ski Time
0,1,3,Schempp,Simon,GER,+1.5,+1.5,NaN,+2.1,8.3,...,men,2017-01-08,1,loop,mass_start,1.5,2016-2017,Oberhof,6.0,NaN
1,2,6,Lesser,Erik,GER,+0.1,+0.1,NaN,+4.4,4.2,...,men,2017-01-08,1,loop,mass_start,0.1,2016-2017,Oberhof,2.0,NaN
2,3,1,Fourcade,Martin,FRA,8:28.7,8:28.7,NaN,+0.7,7.6,...,men,2017-01-08,1,loop,mass_start,0.0,2016-2017,Oberhof,1.0,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,+2.8,+2.8,NaN,+6.0,5.9,...,men,2017-01-08,1,loop,mass_start,2.8,2016-2017,Oberhof,8.0,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,+1.2,+1.2,NaN,+4.7,5.5,...,men,2017-01-08,1,loop,mass_start,1.2,2016-2017,Oberhof,5.0,NaN


## Building predictor's entries

In [24]:
df_predictor['km_to_go'] = df_predictor.apply(lambda x: race_length[x.sex][x.format] - x.distance, axis = 1)
df_loops['km_to_go'] = df_loops.apply(lambda x: race_length[x.sex][x.format] - x.distance, axis = 1)

In [25]:
df_predictor['date'] = list(map(lambda x: x.strip(), df_predictor['date']))

In [26]:
all_dates = df_predictor['date'].unique()
all_dates_datetime = pd.to_datetime(all_dates, format = '%Y-%m-%d')

In [27]:
idx_dates = np.argsort(all_dates_datetime) + 1

dict_date_idx = {all_dates[i]: idx_dates[i] for i in range(len(all_dates))}

In [28]:
df_predictor['date_order'] = df_predictor['date'].map(dict_date_idx)
df_loops['date_order'] = df_loops['date'].map(dict_date_idx)

In [29]:
format_to_ranges ={'individual': ['prone', 'standing', 'prone', 'standing'], 'sprint': ['prone', 'standing'], 'pursuit': ['prone', 'prone', 'standing', 'standing'],\
    'mass_start' : ['prone', 'prone', 'standing', 'standing'], 'sprint_2': ['prone', 'standing'], 'pursuit_2': ['prone', 'prone', 'standing', 'standing']}

In [30]:
def km_to_ranges(km, format, sex):
    tot_km = race_length[sex][format]
    km_per_loop = tot_km/(loops_race[format] + 1)
    shooting_ranges_km = [km_per_loop * i for i in np.arange(loops_race[format]) + 1]

    range_done = sum(km >= np.array(shooting_ranges_km))
    
    n_standing = format_to_ranges[format][range_done:].count('standing')
    n_prone = format_to_ranges[format][range_done:].count('prone')

    return (n_prone, n_standing)


In [31]:
df_predictor[['standing_remaining', 'prone_remaining']] = df_predictor.apply(lambda x: km_to_ranges(x.distance, x.format, x.sex), axis = 1, result_type = 'expand')

In [32]:
df_loops[['standing_remaining', 'prone_remaining']] = df_loops.apply(lambda x: km_to_ranges(x.distance, x.format, x.sex), axis = 1, result_type = 'expand')

In [35]:
df_predictor.columns = ['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Sector Time',
       'Sector Rank', 'Net Time', 'Behind', 'sex', 'distance', 'race',
       'source', 'format', 'season', 'date', 'position', 'venue', 'vRank',
       'Unnamed: 0', 'Day Time', 'km_to_go', 'date_order',
       'standing_remaining', 'prone_remaining']

In [36]:
df_predictor['full_name'] = df_predictor['Given Name'] + '_' + df_predictor['Family Name']

In [38]:
df_loops.columns = ['Rank', 'Bib', 'Family Name', 'Given Name', 'Nation', 'Cumulative Time',
       'Loop Time', 'Course Time\n', 'Course Time', 'Range Time',
       'Shooting Time', 'Penalty Time', 'distance', 'sex', 'date', 'race',
       'source', 'format', 'Behind', 'season', 'venue', 'position', 'Ski Time',
       'km_to_go', 'date_order', 'standing_remaining', 'prone_remaining']

In [39]:
df_loops['full_name'] = df_loops['Given Name'] + '_' + df_loops['Family Name']

In [40]:
df_predictor['full_name'][df_predictor['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/2715052277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictor['full_name'][df_predictor['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'


In [41]:
df_loops['full_name'][df_loops['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_2873/1105425695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loops['full_name'][df_loops['full_name'] == 'Alexandr_Loginov'] = 'Alexander_Loginov'


In [42]:
df_loops['Cumulative Time'] = np.array((map(to_seconds_loop, df_loops['Cumulative Time'])))

In [43]:
df_predictor.full_name.unique()[0]

'Simon_Schempp'

# Percentage standing and percentage shooting

In [53]:
tmp = df_predictor[df_predictor['full_name'] == 'Simon_Schempp']
print(tmp.sex, tmp.)

,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,date,position,vRank,Unnamed: 0,Day Time,km_to_go,date_order,standing_remaining,prone_remaining,full_name
0,1,3,Schempp,Simon,GER,2:10.1,4.0,2:10.1,1.2,men,...,2017-01-08,4.0,NaN,NaN,NaN,14.2,29,2,2,Simon_Schempp
29,1,3,Schempp,Simon,GER,1:18.3,6.0,3:28.4,1.6,men,...,2017-01-08,6.0,NaN,NaN,NaN,13.5,29,2,2,Simon_Schempp
58,1,3,Schempp,Simon,GER,1:26.1,5.0,9:56.3,2.7,men,...,2017-01-08,5.0,NaN,NaN,NaN,11.2,29,1,2,Simon_Schempp
87,1,3,Schempp,Simon,GER,1:15.5,4.0,11:11.8,1.7,men,...,2017-01-08,4.0,NaN,NaN,NaN,10.5,29,1,2,Simon_Schempp
116,1,3,Schempp,Simon,GER,1:29.2,3.0,17:36.0,1.1,men,...,2017-01-08,3.0,NaN,NaN,NaN,8.2,29,0,2,Simon_Schempp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162882,58.0,89.0,Schempp,Simon,GER,32.3,50.0,19:53.0,101.0,men,...,2021-01-08,50.0,NaN,NaN,15:19:23,3.0,148,0,0,Simon_Schempp
162987,58.0,89.0,Schempp,Simon,GER,1:03.6,51.0,20:56.6,112.0,men,...,2021-01-08,51.0,NaN,NaN,15:20:26,2.5,148,0,0,Simon_Schempp
163092,58.0,89.0,Schempp,Simon,GER,3:17.6,54.0,24:14.2,130.0,men,...,2021-01-08,54.0,NaN,NaN,15:23:44,1.3,148,0,0,Simon_Schempp
163197,58.0,89.0,Schempp,Simon,GER,1:03.7,54.0,25:17.9,138.1,men,...,2021-01-08,54.0,NaN,NaN,15:24:47,1.0,148,0,0,Simon_Schempp


In [ ]:
#def behind_to_position(df):
    
    final_columns = np.array([])

    
    unique_races = df['race'].unique()
    
    for race in unique_races:
        df_race = df[df['race'] == race]
        unique_distances = df_race['distance'].unique()

        for distance in unique_distances:
            df_stands = df_race[df_race['distance'] == distance]
            idx_sort_behind = np.argsort(df_stands['Behind'])

            df_stands['position'] = idx_sort_behind + 1
            final_columns = np.append(final_columns, df_stands)

In [ ]:
tmp

In [ ]:
i = 0
for _ in tmp:
    i+=1
    try:
        __ = to_seconds(_)
    except:
        print(_, i)

+2:43.3zy 98290


In [ ]:
tmp = df_predictor['Behind']
tmp = np.array(tmp)

In [ ]:
df_predictor['date_order'] = df_predictor['date_order'].astype({'date_order': 'int32'})
#input_predictor[np.isnan(input_predictor['date_order'])]

In [ ]:
tmp = tmp.sort_values(by = ['sex', 'race', 'distance'])
tmp.reset_index(inplace = True, drop = True)
tmp.to_csv('joint_df.csv', index = False)

In [ ]:
indexes = {'men':int(0), 'women':int(0)}
indexes['men'] += 1
indexes['men']

1

In [ ]:
pd.read_csv('/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/women/2018-2019_w/Hochfilzen/metadata_2_Hochfilzen_pursuit.txt', sep = '\t', index_col = 0)

,Value (in m)
Total Course Length,7590
Height Difference,31.5
Max Climb,28
Total Climb,257
Date,2018-12-15


In [ ]:
input_predictor.reset_index(inplace = True, drop = True)

In [ ]:
input_predictor

,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,distance,race,date,Unnamed: 8,Unnamed: 0,vRank,Unnamed: 9,Day Time
0,1,3,Schempp,Simon,GER,2:10.1,4,2:10.1,1.2,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
1,2,6,Lesser,Erik,GER,2:11.0,7,2:11.0,2.1,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
2,3,1,Fourcade,Martin,FRA,2:08.9,1,2:08.9,0.0,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
3,4,16,Beatrix,Jean Guillaume,FRA,2:15.5,23,2:15.5,6.6,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
4,5,8,Bjoerndalen,Ole Einar,NOR,2:11.3,8,2:11.3,2.4,men,0.8,1.0,2017-01-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,78,Sabule,Annija,LAT,46.4,70,23:27.5,+3:52.8,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:17:26
71,72,66,Colebourn,Jillian Wei-Lin,AUS,46.4,72,24:26.9,+4:52.2,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:12:25
72,73,64,Bulina,Sanita,LAT,55.4,73,24:40.4,+5:05.7,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:11:40
73,74,50,Tanaka,Yurie,JPN,44.8,74,24:46.6,+5:11.9,women,6.9,148.0,2022-03-18,NaN,NaN,NaN,NaN,14:04:45


In [ ]:
tt = pd.read_csv('joint_df.csv')
tt[tt['Family Name'] == 'Loginov']

/var/folders/92/pt64shtn3fx70tf0m4ng91240000gn/T/ipykernel_799/731202880.py:1: DtypeWarning: Columns (0,1,6,14,15,17,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tt = pd.read_csv('joint_df.csv')


,Rank,Bib,Family Name,Given Name,Nation,Sector Time,Sector Rank,Net Time,Behind,sex,...,Unnamed: 9,Day Time,Cumulative Time,Loop Time,Course Time,Course Time.1,Range Time,Shooting Time,Penalty Time,Ski Time
16473,72,31.0,Loginov,Alexander,RUS,1:41.2,26.0,1:41.2,4.5,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16573,72,31.0,Loginov,Alexander,RUS,48.9,19.0,2:30.1,4.4,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673,72,31.0,Loginov,Alexander,RUS,3:08.8,32.0,5:38.9,14.7,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16773,72,31.0,Loginov,Alexander,RUS,1:09.9,31.0,6:48.8,+18.2,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16873,72,31.0,Loginov,Alexander,RUS,2:04.5,96.0,14:39.5,+3:34.9,men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321608,2,32,Loginov,Alexandr,RUS,1:38.2,4.0,17:37.7,+40.9,women,...,NaN,15:02:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321825,2,32,Loginov,Alexandr,RUS,2:18.6,8.0,22:31.2,+57.0,women,...,NaN,15:07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321936,2,32,Loginov,Alexandr,RUS,1:38.1,8.0,24:09.3,+56.7,women,...,NaN,15:09:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322100,2,32,Loginov,Alexandr,RUS,2:15.0,2.0,28:37.8,+22.0,women,...,NaN,15:13:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tmp ='/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Anterselva/metadata_2_Anterselva_mass_start.txt'
pd.read_csv(tmp, sep = '\t', index_col = 0).T['Date'][0]

'2017-01-22'

In [ ]:
np.sort(np.array(tmp))

array(['/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/1.0km.txt',
       '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Oberhof/intermediates/mass_start/3.8km.txt'],
      dtype='<U113')

In [ ]:
Hochfilzen pursuit men 2016-2017